In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv('/kaggle/input/top-beer-information/beer_data_set.csv')

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df.isnull().sum()

In [ ]:
sns.heatmap(df.corr())

In [ ]:
df.describe()

In [ ]:
df.hist(figsize=(15,15))

First, I tried LinerRegression and also tried Logarithmic of features.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression,Ridge,LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
x=df.drop(['Name', 'key', 'Style','Brewery','Description','Ave Rating'],axis=1)
y=df['Ave Rating']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=0.8,random_state=0)

In [ ]:
model_lr=LinearRegression()
model_lr.fit(x_train,y_train)
pred_train=model_lr.predict(x_train)
pred_test=model_lr.predict(x_test)
print("train_RMSE:",np.sqrt(mean_squared_error(y_train, pred_train)))
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred_test)))
print("train_MAE:",mean_absolute_error(y_train, pred_train))
print("test_MAE:",mean_absolute_error(y_test, pred_test))
print("R^2:{}".format(model_lr.score(x_test, y_test)))

In [ ]:
from scipy.stats import skew

In [ ]:
AP=pd.DataFrame({'ap':y,'log(ap+1)':np.log1p(y)})
print(AP, '¥n')

print('ap skew        :',skew(AP['ap']))
print('log(ap+1) skew:', skew(AP['log(ap+1)']))

AP.hist()

In [ ]:
x_skew=x.apply(lambda x:skew(x))
print(x_skew)

The features which skews are over 0.75 should be logarithmic.

In [ ]:
x_skew = x_skew[x_skew > 0.75]
print('-----Skewness greater than 0.75-----')
print(x_skew)
x_skew = x_skew.index

x[x_skew] = np.log1p(x[x_skew])
x[x_skew]

In [ ]:
x

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=0.8,random_state=0)

In [ ]:
model_lr=LinearRegression()

In [ ]:
model_lr.fit(x_train,y_train)
pred_train=model_lr.predict(x_train)
pred_test=model_lr.predict(x_test)
print("train_RMSE:",np.sqrt(mean_squared_error(y_train, pred_train)))
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred_test)))
print("train_MAE:",mean_absolute_error(y_train, pred_train))
print("test_MAE:",mean_absolute_error(y_test, pred_test))
print("R^2:{}".format(model_lr.score(x_test, y_test)))

The Logarithmic is not usefull to improve in LinerRegression.

Second, I tried Ridge and LassoCV.

In [ ]:
x=df.drop(['Name', 'key', 'Style','Brewery','Description','Ave Rating'],axis=1)
y=df['Ave Rating']
x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=0.8,random_state=0)

In [ ]:
def rmse_cv(model):
    rmse = np.sqrt(
        -cross_val_score(
            model, x_train, y_train,
            scoring="neg_mean_squared_error", 
            cv = 5))
    return(rmse)

In [ ]:
model_rg = Ridge()

alphas = [3.2,3.4, 3.6, 3.8,4.0,4.2]
cv_rg = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_rg = pd.Series(cv_rg, index = alphas)

print('Ridge RMSE loss:')
print(cv_rg, '\n')

print('Ridge RMSE loss Mean:')
print(cv_rg.mean())


plt.ﬁgure(ﬁgsize=(10, 5))
plt.plot(cv_rg)
plt.grid()
plt.title('Validation - by regularization strength')
plt.xlabel('Alpha')
plt.ylabel('RMSE')
plt.show()

In [ ]:
model_rg.fit(x_train,y_train)
pred1=model_rg.predict(x_test)
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred1)))
print("test_MAE:",mean_absolute_error(y_test, pred1))
print("R^2:{}".format(model_rg.score(x_test, y_test)))

In [ ]:
model_ls = LassoCV(
    alphas = [1, 0.1, 0.001, 0.0005]).fit(x_train, y_train)

print('Lasso regression RMSE loss:')
print(rmse_cv(model_ls))

print('Average loss:', rmse_cv(model_ls).mean())
print('Minimum loss:', rmse_cv(model_ls).min())
print('Best alpha  :', model_ls.alpha_) 


In [ ]:
model_ls.fit(x_train,y_train)
pred2=model_ls.predict(x_test)
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred2)))
print("test_MAE:",mean_absolute_error(y_test, pred2))
print("R^2:{}".format(model_ls.score(x_test, y_test)))

Ridge and LassoCV were not usefull to improve.

Third, I tried XGBoost.

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(x_train, label = y_train)

params = {"max_depth":3, "eta":0.1}

cross_val = xgb.cv(
    params,
    dtrain,
    num_boost_round=1000,
    early_stopping_rounds=50)
cross_val

In [ ]:
plt.ﬁgure(ﬁgsize=(8, 6))
plt.plot(cross_val.loc[0:,["test-rmse-mean", "train-rmse-mean"]])
plt.grid()
plt.xlabel('num_boost_round')
plt.ylabel('RMSE')
plt.show()

In [ ]:
model_xgb = xgb.XGBRegressor(
    n_estimators=325,
    max_depth=3,
    learning_rate=0.1)
model_xgb.fit(x_train, y_train)
pred3=model_xgb.predict(x_test)

print('xgboost RMSE loss:')
print(rmse_cv(model_xgb).mean())
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred3)))
print("test_MAE:",mean_absolute_error(y_test, pred3))
print("R^2:{}".format(model_xgb.score(x_test, y_test)))

Optimization of Pamaeterss.

In [ ]:
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest=xgb.DMatrix(x_test, label = y_test)

In [ ]:
base_params = {
    'booster': 'gbtree',
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
}

watchlist = [(dtrain, 'train'), (dtest, 'eval')]

In [ ]:
import optuna
from sklearn.metrics import r2_score
import copy

tmp_params = copy.deepcopy(base_params)

def optimizer(trial):
    eta = trial.suggest_uniform('eta', 0.01, 0.3)
    max_depth = trial.suggest_int('max_depth', 4, 15)
    __lambda = trial.suggest_uniform('lambda', 0.7, 2)

    tmp_params['eta'] = eta
    tmp_params['max_depth'] = max_depth
    tmp_params['lambda'] = __lambda

    model = xgb.train(tmp_params, dtrain, num_boost_round=50)
    predicts = model.predict(dtest)

    r2 = r2_score(y_test, predicts)
    print(f'#{trial.number}, Result: {r2}, {trial.params}')

    return r2

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(optimizer, n_trials=500)

In [ ]:
study.best_params

In [ ]:
paramas=study.best_params

In [ ]:
from sklearn.metrics import r2_score

def eval_model(params, dtrain, dtest):
    model = xgb.train(params, dtrain, num_boost_round=100, verbose_eval=False, evals=watchlist)
    predicts = model.predict(dtest)
    r2 = r2_score(y_test, predicts)

    return r2

In [ ]:
base_r2 = eval_model(base_params, dtrain, dtest)

merged_params = dict(base_params, **study.best_params)
best_r2 = eval_model(merged_params, dtrain, dtest)

print(f'Base params: {base_params}')
print(f'Best params: {merged_params}')
print(f'Base: {base_r2}, Best: {best_r2}, Diff: {best_r2 - base_r2}')

Next, I tried  CatBoost.

In [ ]:
from catboost import CatBoostRegressor, FeaturesData, Pool

In [ ]:
categorical_features_indices = np.where(x.dtypes != np.float)[0]

In [ ]:
model_cat = CatBoostRegressor(iterations=2000, learning_rate=0.05, depth=5)
model_cat.fit(x_train, y_train)
pred4=model_cat.predict(x_test)

print('catboost RMSE loss:')
print(rmse_cv(model_cat).mean())
print("test_RMSE:",np.sqrt(mean_squared_error(y_test, pred4)))
print("test_MAE:",mean_absolute_error(y_test, pred4))
print("R^2:{}".format(model_cat.score(x_test, y_test)))

I tried to tune the parameters, but I could not improve the results.
But by Catboost, I got MSE 0.26, MAE 0.19 and R2 0.66.

Last one is PCA and Clustering.

In [ ]:
df1=df.drop(['Name', 'Style','Brewery','Description','key'], axis=1)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca.fit(df1)

In [ ]:
df1_2d = pca.transform(df1)

df1_2d = pd.DataFrame(df1_2d)
df1_2d.index = df1.index
df1_2d.columns = ['PC1','PC2','PC3','PC4']
df1_2d.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
components=pd.DataFrame(pca.components_)
components.columns=df1.columns
components.index=['PCA1','PCA2','PCA3','PCA4']
components

In [ ]:
components.plot.bar(figsize=(15,8))
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

In [ ]:
from sklearn.cluster import KMeans   
kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit(df1)

df1_2d['cluster'] = pd.Series(clusters.labels_, index=df1_2d.index)

In [ ]:
df1_2d.plot(
        kind='scatter',
        x='PC2',y='PC1',
        c=df1_2d.cluster.astype(np.float),
        figsize=(16,8))

In [ ]:
df1['cluster']=df1_2d['cluster']
df1

In [ ]:
df1.groupby('cluster')['cluster'].count().plot.pie(figsize=(8,8),autopct="%.1f%%")

In [ ]:
df1.groupby('cluster').mean()